![](https://wherobots.com/wp-content/uploads/2023/12/Inline-Blue_Black_onWhite.png)
# Unity Catalog Delta Tables Example
In this example, we will:
- Load data from a Delta table from a foreign catalog backed by Databricks Unity Catalog
- Transform the data using Spatial SQL
- Write the transformed data to a new Iceberg table, which can either be in Databricks Unity Catalog or in a Wherobots-managed catalog

# Before You Begin
This example assumes you have already created a Delta-format foreign catalog connection to Databricks Unity Catalog, which contains a Delta table to operate on. This Delta table must contain a geometry column with values encoded in WKT format.

If you plan to write the data to an Iceberg table in Unity Catalog, you will need another Iceberg-format foreign catalog connection.

Fill in the values in the cell below with the details of the table to operate on.

In [ ]:
catalog = "<your_catalog>"
namespace = "<your_namespace>"
table = "<your_table>"
geometry_column = "<your_geometry_column>"

# Define Sedona Context

In [ ]:
from sedona.spark import *

config = SedonaContext.builder().getOrCreate()
sedona = SedonaContext.create(config)

# Load and Process the Data

1. Define an area of interest that we will use to filter the results
2. Load the data from the Unity Catalog table and create a new `havasu_geometry` column where the WKT value of the original geometry column has been converted to a Havasu geometry type.
3. Create a new filtered data set that contains only the rows from the original table where the geometry intersects the area of interest

In [ ]:
from pyspark.sql import functions as f
import json

aoi = json.dumps({
  "type": "Feature", 
  "properties": {},
  "geometry": {
    "type": "Polygon", 
    "coordinates": [
      [
        [-87.69953273369673,41.96159824365796],
        [-87.69953273369673,41.94631973053055],
        [-87.6741283563407,41.94631973053055],
        [-87.6741283563407,41.96159824365796],
        [-87.69953273369673,41.96159824365796],
      ]
    ]
  }
})

all_results_df = sedona.sql(f"SELECT * FROM `{catalog}`.`{namespace}`.`{table}`").withColumn("havasu_geometry", f.expr(f"ST_GEOMFROMTEXT({geometry_column})"))
filtered_results_df = all_results_df.where(f"ST_INTERSECTS(havasu_geometry, ST_GeomFromGeoJSON('{aoi}'))")
filtered_results_df.show(10)

# Write the Data to a Unity Catalog Table

Because Databricks does not support writing to Delta tables from external workloads, we will write the transformed data to an Iceberg table.

Fill in the value of `iceberg_catalog` before running this cell.

1. Call `cache()` and `count()` on the Delta table results dataframe, so that we have a pinned copy of the data that cannot be modified by a concurrent write to the underlying table.
2. Drop the `havasu_geometry` column because the Havasu geometry type will not be recognized by Unity Catalog
3. Write the filtered data to a new table

In [ ]:
iceberg_catalog = "<your_iceberg_catalog>"

filtered_results_df.cache()
filtered_results_df.count()

filtered_results_df \
  .drop("havasu_geometry") \
  .writeTo(f"`{iceberg_catalog}`.`{namespace}`.`{table}_filtered_from_delta`") \
  .createOrReplace()

# Write the Data to a Wherobots-managed Catalog Table

1. Create a database in the default `org_catalog` for the new table
2. Call `cache()` and `count()` on the Delta table results dataframe, so that we have a pinned copy of the data that cannot be modified by a concurrent write to the underlying table.
3. Replace the original WKT column with the new Havasu geometry type column. Storing the data in Havasu format makes further processing more efficient
4. Write the filtered data to a new table

In [ ]:
sedona.sql("CREATE DATABASE IF NOT EXISTS org_catalog.unity_catalog_examples")

filtered_results_df.cache()
filtered_results_df.count()

filtered_results_df \
  .drop(geometry_column) \
  .withColumnRenamed("havasu_geometry", geometry_column) \
  .writeTo(f"org_catalog.unity_catalog_examples.`{table}_filtered_from_delta`") \
  .createOrReplace()